In [126]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.decomposition import NMF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, hamming_loss
import joblib

# Configuration de l'affichage pour voir toutes les colonnes
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Librairies chargées avec succès !")

Librairies chargées avec succès !


In [127]:
# Chargement de la matrice et des catalogues
print("--- Chargement des données ---")
X = load_npz('../out/user_permission_matrix_sparse.npz')
users_df = pd.read_csv('../out/users_catalog.csv')
perms_df = pd.read_csv('../out/perm_catalog.csv')
apps_df = pd.read_csv('../out/app_catalog.csv')

print(f"Matrice chargée : {X.shape} (Utilisateurs x Permissions)")
print(f"Nombre d'utilisateurs : {len(users_df)}")
print(f"Nombre de permissions : {len(perms_df)}")

--- Chargement des données ---
Matrice chargée : (5000, 324) (Utilisateurs x Permissions)
Nombre d'utilisateurs : 5000
Nombre de permissions : 324


In [128]:
# --- Configuration NMF ---

# 1. Calcul de la fréquence
X_binary = (X > 0).astype(int)
user_count_per_perm = np.array(X_binary.sum(axis=0)).flatten()
total_users = X.shape[0]
freq_per_perm = user_count_per_perm / total_users

# 2. Définition des bornes
# On garde le seuil agressif pour virer le bruit (Slack, Zoom...)
UPPER_LIMIT = 0.7  # Si + de 10% l'ont, on supprime...
LOWER_LIMIT = 0.00  # On garde toutes les raretés


# 3. Application du masque combiné
# Règle : (Est Rare) OU (Est Protégé)
# Si c'est fréquent MAIS protégé, on garde !
perms_to_keep_mask = ((freq_per_perm <= UPPER_LIMIT) & (freq_per_perm >= LOWER_LIMIT))

# Stats pour vérifier
n_total = len(perms_df)
n_kept = np.sum(perms_to_keep_mask)

print(f"Permissions totales : {n_total}")
print(f"Permissions conservées pour la NMF : {n_kept}")
print(f"Permissions rejetées car trop communes (> {UPPER_LIMIT*100}%) : {np.sum(freq_per_perm > UPPER_LIMIT)}")



# 5. Filtrage et Lancement NMF
X_filtered = X[:, perms_to_keep_mask]
perms_df_filtered = perms_df[perms_to_keep_mask].reset_index(drop=True)

N_ROLES = 8
print(f"\nLancement NMF sur {X_filtered.shape[1]} permissions...")
model_nmf = NMF(n_components=N_ROLES, init='nndsvd', random_state=42, max_iter=1000)
W = model_nmf.fit_transform(X_filtered)

# Normalisation
W_norm = W / W.sum(axis=1, keepdims=True)
W_norm = np.nan_to_num(W_norm)

# Mise à jour pour la suite
print("NMF terminée !")
perms_df = perms_df_filtered 
H = model_nmf.components_ # Important pour l'affichage des noms

Permissions totales : 324
Permissions conservées pour la NMF : 324
Permissions rejetées car trop communes (> 70.0%) : 0

Lancement NMF sur 324 permissions...
NMF terminée !


In [129]:
# --- CELLULE D'ANALYSE DES RÔLES ---

def inspecter_roles(H_matrix, n_top=10):
    print(f"--- ANALYSE DES {len(H_matrix)} RÔLES DÉCOUVERTS ---\n")
    
    for role_id, weights in enumerate(H_matrix):
        # On trie les permissions par poids (les plus importantes d'abord)
        top_indices = weights.argsort()[::-1][:n_top]
        
        # On récupère les détails
        top_perms = perms_df.iloc[top_indices].copy()
        top_perms['poids'] = weights[top_indices]
        
        # On ajoute le nom de l'appli pour y voir clair
        # (Attention de bien avoir 'app_name' dans apps_df)
        details = pd.merge(top_perms, apps_df, on='application_id', how='left')
        
        print(f"🔹 RÔLE ID {role_id}")
        
        # On affiche les 5-6 applis dominantes pour résumer
        apps_dominantes = details['app_name'].unique()[:4]
        print(f"   Dominante : {', '.join(apps_dominantes)}")
        
        # On affiche le détail des permissions
        print(details[['perm_name', 'app_name', 'poids']].to_string(index=False, header=False))
        print("-" * 50)

# Lancement de l'inspection
inspecter_roles(H)

--- ANALYSE DES 8 RÔLES DÉCOUVERTS ---

🔹 RÔLE ID 0
   Dominante : GitHub Enterprise, CrowdStrike Falcon, ServiceNow ITSM, Confluence
  manage_pipeline  GitHub Enterprise 3.738366
            login CrowdStrike Falcon 3.727194
      open_ticket    ServiceNow ITSM 3.720665
manage_identities CrowdStrike Falcon 3.712883
  manage_pipeline         Confluence 3.710987
            login  GitHub Enterprise 3.710656
    open_employee         Talentsoft 3.700795
    read_messages         SharePoint 0.320545
   create_channel         SharePoint 0.307191
      manage_team               Zoom 0.287836
--------------------------------------------------
🔹 RÔLE ID 1
   Dominante : Workday HCM, Talentsoft, Sage Paie, Oracle Financials Cloud
            login             Workday HCM 3.251325
    edit_employee              Talentsoft 3.239916
            login               Sage Paie 3.239462
    open_employee              Talentsoft 3.225341
    edit_employee             Workday HCM 3.223494
export_hr_rep

In [130]:
# --- DÉFINITION HUMAINE DES RÔLES ---
# Remplace les noms ci-dessous par ton analyse

ROLE_NAMES = {
    0: "pack IT",
    1: "RH / Finance",
    2: "Finance",
    3: "Data science",
    4: "DEV OPS Secu",
    5: "Cyber secu",
    6: "ITSM",
    7: "pack Collab",
   
    
}

# --- MISE À JOUR DE LA FONCTION D'AFFICHAGE ---
# On écrase l'ancienne fonction describe_role pour utiliser les nouveaux noms

def describe_role(role_id):
    # On cherche le nom dans ton dictionnaire
    # Si on ne trouve pas, on met un nom par défaut
    custom_name = ROLE_NAMES.get(role_id, f"Rôle {role_id} (Non nommé)")
    return f"[{role_id}] {custom_name}"

print("✅ Noms des rôles enregistrés ! Le modèle parlera maintenant français.")

✅ Noms des rôles enregistrés ! Le modèle parlera maintenant français.


In [131]:
# --- DÉFINITION DU SEUIL ---
# Si un utilisateur a plus de 10% de son "ADN" dans un rôle, on considère qu'il l'a.
THRESHOLD = 0.15

# Y_binary est une matrice de 0 et 1.
# Chaque ligne correspond à un utilisateur, chaque colonne à un rôle.
# Exemple : [1, 0, 0, 1, ...] signifie "Possède le Rôle 0 et le Rôle 3"
Y_binary = (W_norm > THRESHOLD).astype(int)

# Compte le nombre d'utilisateurs pour chaque rôle
roles_counts = Y_binary.sum(axis=0)

print("--- Distribution des Rôles ---")
for i, count in enumerate(roles_counts):
    percentage = (count / len(Y_binary)) * 100
    status = "⚠️ TROP RARE" if percentage < 1 else "✅ OK"
    print(f"Rôle {i} : {count} utilisateurs ({percentage:.2f}%) {status}")

# Vérification statistique
avg_roles = np.mean(np.sum(Y_binary, axis=1))
print(f"--- Statistiques Multi-Rôles ---")
print(f"Avec un seuil de {THRESHOLD*100}%, un utilisateur possède en moyenne {avg_roles:.2f} rôles.")
print("Si ce chiffre est proche de 1, baisse le seuil. S'il est > 5, monte le seuil.")

--- Distribution des Rôles ---
Rôle 0 : 1326 utilisateurs (26.52%) ✅ OK
Rôle 1 : 504 utilisateurs (10.08%) ✅ OK
Rôle 2 : 559 utilisateurs (11.18%) ✅ OK
Rôle 3 : 584 utilisateurs (11.68%) ✅ OK
Rôle 4 : 1010 utilisateurs (20.20%) ✅ OK
Rôle 5 : 1141 utilisateurs (22.82%) ✅ OK
Rôle 6 : 1120 utilisateurs (22.40%) ✅ OK
Rôle 7 : 427 utilisateurs (8.54%) ✅ OK
--- Statistiques Multi-Rôles ---
Avec un seuil de 15.0%, un utilisateur possède en moyenne 1.33 rôles.
Si ce chiffre est proche de 1, baisse le seuil. S'il est > 5, monte le seuil.


In [132]:
print("--- Préparation et Entraînement Supervisé ---")

# 1. Encodage des entrées (X)
le_dept = LabelEncoder()
users_df['dept_encoded'] = le_dept.fit_transform(users_df['department'])

le_pos = LabelEncoder()
users_df['pos_encoded'] = le_pos.fit_transform(users_df['position'])



# L'IA doit juger uniquement sur le "Qui je suis" (Poste/Dept), pas le "Où je suis".
X_features = users_df[['dept_encoded', 'pos_encoded']] 


X_train, X_test, y_train, y_test = train_test_split(X_features, Y_binary, test_size=0.2, random_state=42)

# Entraînement...
clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced') # On garde balanced
clf.fit(X_train, y_train)

# 4. Évaluation rapide
y_pred = clf.predict(X_test)
# L'accuracy exacte est sévère (il faut avoir TOUT bon sur la ligne), donc on regarde un score moyen
print(f"Modèle entraîné sur {len(X_train)} utilisateurs.")
print(f"Précision 'Subset' (Tout bon ou rien) : {accuracy_score(y_test, y_pred):.2%}")
print("--- Rapport de Performance ---")
print(classification_report(y_test, y_pred))
print(f"Hamming Loss (Taux d'erreur par étiquette) : {hamming_loss(y_test, y_pred):.2%}")
print("(Exemple : 2% signifie que 98% des cases cochées/décochées sont justes !)")

--- Préparation et Entraînement Supervisé ---
Modèle entraîné sur 4000 utilisateurs.
Précision 'Subset' (Tout bon ou rien) : 63.40%
--- Rapport de Performance ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       262
           1       1.00      1.00      1.00        97
           2       1.00      0.96      0.98       113
           3       1.00      0.92      0.96       105
           4       0.72      0.99      0.83       187
           5       0.83      0.99      0.90       241
           6       0.84      0.74      0.79       235
           7       0.09      0.20      0.12        85

   micro avg       0.78      0.89      0.83      1325
   macro avg       0.81      0.85      0.82      1325
weighted avg       0.84      0.89      0.86      1325
 samples avg       0.86      0.92      0.86      1325

Hamming Loss (Taux d'erreur par étiquette) : 5.99%
(Exemple : 2% signifie que 98% des cases cochées/décochées sont justes !)


In [135]:
# --- DÉMO PROBABILISTE ---

print("--- MOTEUR DE RECOMMANDATION (IA + RÈGLES) ---")
NEW_DEPT = "Department IT"
NEW_POS  = "DevOps"
NEW_LOC  = "Lyon"

print(f"Profil : {NEW_POS} / {NEW_DEPT} / {NEW_LOC}")
print("-" * 40)

# ANALYSE (MÉTIER)
# On abaisse le seuil de décision pour être pro-actif
CONFIDENCE_THRESHOLD = 0.25  # Si l'IA est sûre à 25%, on propose le rôle !

try:
    # On n'encode QUE Dept et Pos (On a retiré la Loc de l'IA)
    input_features = [[
        le_dept.transform([NEW_DEPT])[0],
        le_pos.transform([NEW_POS])[0]
    ]]
    
    # Au lieu de predict(), on récupère les probabilités brutes
    # Random Forest renvoie une liste de tableaux (un par rôle)
    # C'est un peu technique, mais voici la formule magique :
    probas = np.array(clf.predict_proba(input_features)) 
    # probas a la forme (n_roles, n_samples, 2_classes) -> On veut juste la proba du "1" (Oui)
    scores_par_role = probas[:, 0, 1] 
    
    found_roles = []
    print(f" [IA] Analyse des besoins métier (Seuil > {CONFIDENCE_THRESHOLD*100}%) :")
    
    # On parcourt tous les rôles et leur score
    role_detected = False
    for r_id, score in enumerate(scores_par_role):
        if score >= CONFIDENCE_THRESHOLD:
            role_detected = True
            # On affiche une barre de confiance visuelle
            bar = "█" * int(score * 10)
            print(f"   [{bar:<10}] {score:.0%} de confiance -> {describe_role(r_id)}")
            
    if not role_detected:
        print("   (Aucun signal métier fort détecté)")

except Exception as e:
    print(f"Erreur IA : {e}")

# 2. RÈGLES (LOGISTIQUE)
print(f"\n [RÈGLES] Droits automatiques :")
if NEW_LOC == "Lyon":
    print("    Badge Lyon")
    
elif NEW_LOC == "Paris":
    print("    Badge Paris")

print("-" * 40)

--- MOTEUR DE RECOMMANDATION (IA + RÈGLES) ---
Profil : DevOps / Department IT / Lyon
----------------------------------------
 [IA] Analyse des besoins métier (Seuil > 25.0%) :
   [██████████] 100% de confiance -> [0] pack IT
   [█████     ] 58% de confiance -> [7] pack Collab

 [RÈGLES] Droits automatiques :
    Badge Lyon
----------------------------------------


/Users/tidianetall/Desktop/Desktop/Cours/PCE/Wavestone/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
